In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# Baixar os dados do Kaggle (certifique-se de ter o kaggle.json configurado)
os.system("kaggle datasets download -d olistbr/brazilian-ecommerce -p ./ --unzip")


1

In [2]:

folder = r"C:\Users\helio\OneDrive\Documentos\Insper\PADS\PADS\Trimestre-3\MÉTODOS DE DESIGN PARA CIÊNCIA DE DADOS\Olist_Data"
# Carregar os datasets
orders = pd.read_csv(os.path.join(folder+"/olist_orders_dataset.csv"))
order_items = pd.read_csv(os.path.join(folder+"/olist_order_items_dataset.csv"))
customers = pd.read_csv(os.path.join(folder+"/olist_customers_dataset.csv"))
sellers = pd.read_csv(os.path.join(folder+"/olist_sellers_dataset.csv"))
products = pd.read_csv(os.path.join(folder+"/olist_products_dataset.csv"))
payments = pd.read_csv(os.path.join(folder+"/olist_order_payments_dataset.csv"))
reviews = pd.read_csv(os.path.join(folder+"/olist_order_reviews_dataset.csv"))
geolocation = pd.read_csv(os.path.join(folder+"/olist_geolocation_dataset.csv"))


In [ ]:
import pandas as pd
import os
import numpy as np
import plotly.express as px
import imageio.v2 as imageio  # Para criar o GIF

# Carregar datasets
folder = r"C:\Users\helio\OneDrive\Documentos\Insper\PADS\PADS\Trimestre-3\MÉTODOS DE DESIGN PARA CIÊNCIA DE DADOS\Olist_Data"

orders = pd.read_csv(os.path.join(folder, "olist_orders_dataset.csv"))
order_items = pd.read_csv(os.path.join(folder, "olist_order_items_dataset.csv"))
products = pd.read_csv(os.path.join(folder, "olist_products_dataset.csv"))

# Converter timestamp para data e extrair ano e mês
orders["order_purchase_timestamp"] = pd.to_datetime(orders["order_purchase_timestamp"])
orders["year_month"] = orders["order_purchase_timestamp"].dt.year + (orders["order_purchase_timestamp"].dt.month - 1) / 12

# Unir tabelas
df = order_items.merge(orders[["order_id", "year_month"]], on="order_id", how="left")
df = df.merge(products[["product_id", "product_length_cm", "product_height_cm", "product_width_cm"]], on="product_id", how="left")

# Calcular volume (cm³)
df["volume_cm3"] = df["product_length_cm"] * df["product_height_cm"] * df["product_width_cm"]

# Filtrar dados válidos
df = df.dropna(subset=["volume_cm3", "freight_value", "year_month"])

# Aplicar amostragem para reduzir tempo de execução (aumentei para 1% dos dados)
df = df.sample(frac=0.01, random_state=42)

# Criar lista de frames interpolados (1 frame por mês)
frames = np.arange(df["year_month"].min(), df["year_month"].max(), 1/12)

# Criar pasta para armazenar imagens temporárias
output_folder = "frames_gif"
os.makedirs(output_folder, exist_ok=True)

# Lista para armazenar os caminhos das imagens
image_paths = []

# Criar gráficos individuais para cada frame
for i, year in enumerate(frames):
    if i % 10 == 0:  # Reduz a frequência de prints para não travar o notebook
        print(f"Gerando frame {i}/{len(frames)} - Ano: {year:.2f}")
    
    df_temp = df[df["year_month"] <= year]  # Filtrar os dados até o mês atual
    
    fig = px.scatter(df_temp, x="volume_cm3", y="freight_value",
                     title=f"Correlação entre Volume e Preço do Frete ({year:.2f})",
                     labels={"volume_cm3": "Volume do Produto (cm³)", "freight_value": "Preço do Frete (R$)"},
                     opacity=0.6)
    
    # Caminho do arquivo de imagem
    img_path = os.path.join(output_folder, f"frame_{i:03d}.png")
    fig.write_image(img_path, scale=1.5)  # Salva com qualidade otimizada
    image_paths.append(img_path)

# Criar GIF a partir das imagens geradas (ajustada a duração para melhor fluidez)
gif_path = "grafico_animado.gif"
with imageio.get_writer(gif_path, mode='I', duration=0.2) as writer:
    for img in image_paths:
        image = imageio.imread(img)
        writer.append_data(image)

# Exibir GIF no Jupyter Notebook
from IPython.display import Image
Image(gif_path)

Gerando frame 0/22 - Ano: 2016.75
